In [1048]:
# Install necessary packages (uncomment and run if not already installed)
# !pip install torch torchvision torchaudio transformers librosa soundfile numpy openpyxl scikit-learn pandas pydub opensmile

# Import necessary libraries
import os
import re
import glob
import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import mysql.connector
import mysql.connector
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2Model,
    WhisperProcessor,
    WhisperModel,
    Wav2Vec2ForSequenceClassification,
)
from pydub import AudioSegment
import opensmile
import warnings
from sklearn.neighbors import NearestNeighbors
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:

# Establish connection to MySQL database
def connect_to_db():
    connection = mysql.connector.connect(
        host='localhost',
        user='*****************',
        password='******************',
        database='Audio_features'
    )
    return connection

In [1050]:
Oldid_newid_file = '/home/karthik9/Klaatch Sanitized TextCEL 2021 to 2023 (1).xlsx'
Oldid_newid = pd.read_excel(Oldid_newid_file,sheet_name=3)
Oldid_newid = Oldid_newid.dropna(subset=['New ID'])
Oldid_newid['New ID'] = Oldid_newid['New ID'].astype(int)
Oldid_newid['Old ID'] = Oldid_newid['Old ID'].astype(int)

# Drop duplicate New IDs to ensure uniqueness
Oldid_newid = Oldid_newid.drop_duplicates(subset=['New ID'], keep='first')

# Create dictionary for mapping
old_id_mapping = dict(zip(Oldid_newid['New ID'], Oldid_newid['Old ID']))
Oldid_newid

,Old ID,New ID
0,21,938648
1,23,938899
2,24,938705
3,25,938752
4,28,938819
...,...,...
413,904,1022050
414,905,1023055
415,906,1026781
416,907,1026772


In [1051]:
demographics_file = '/sandata/karthik9/Demographics_Klaatch Sanitized TextCEL 2021 to 2023.xlsx'
demographics_df = pd.read_excel(demographics_file,sheet_name=2)
print(len(demographics_df))
demographics_df =  demographics_df[

    demographics_df['Gender'].notna() &
    demographics_df['Race'].notna()
]

demographics_df['New ID'] = demographics_df['New ID'].replace("NONE", np.nan)
demographics_df['Old ID'] = demographics_df['Old ID'].astype(str)
demographics_df["Old ID"] =  demographics_df["Old ID"].replace("NONE", np.nan)
demographics_df['Old ID']



438


0      419
1      876
2      818
3      117
4      517
      ... 
429    NaN
430    NaN
431    NaN
432    NaN
433    NaN
Name: Old ID, Length: 417, dtype: object

In [1052]:
transcripts_df = pd.read_csv('/sandata/karthik9/Klaatch_transcripts.csv')

In [1053]:
transcripts_df

,New ID,Old ID,Recording File Name,Text,CEL Total,CELVAL1,CELVAL2,CELVAL3,CEL1,CEL2,...,Extraversion,Agreeable,Neuroticism,Unnamed: 22,SQ risk level,Previous SQ,Previous SQ risk Level,Engage1 (2023),Engage2 (2023),New E# (2023)
0,NaN,559.0,559_2021-01-22,"Hello. OK. Sure. Yesterday, I went to an appoi...",5.0,1,0,4,Agree,Strongly Agree,...,48.46819,61.65434,46.74303,NaN,High Risk,NaN,No Score,NaN,NaN,NaN
1,938730.0,66.0,66_2021-01-26,Hello. Please leave a message after the tone. ...,3.0,1,1,1,Agree,Agree,...,63.00155,73.50541,48.84080,NaN,High Risk,NaN,No Score,NaN,NaN,NaN
2,938936.0,340.0,340_2021-01-26,"Yes. I'm fine, how are you? Ok. Well, what I u...",2.0,1,0,1,Agree,Strongly Agree,...,38.25154,50.96499,50.57849,NaN,At Risk,NaN,No Score,NaN,NaN,NaN
3,NaN,343.0,343_2021-01-26,"Hello. All right. I'll do it now. Sure. No, no...",1.0,1,0,0,Agree,Strongly Agree,...,44.59551,55.89482,50.66619,NaN,At Risk,NaN,No Score,NaN,NaN,NaN
4,938879.0,383.0,383_2021-01-27,"Hello. Hi, I'm. Ok. OK. Yes, okay. I wake up, ...",5.0,1,3,1,Agree,Disagree,...,61.34613,70.95884,48.78890,NaN,No Risk,NaN,No Score,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,941934.0,NaN,941934_2024-11-07,NaN,1.0,0,0,1,Strongly Agree,Strongly Agree,...,54.45462,NaN,43.86910,NaN,No Risk,NaN,NaN,More than 10 times a week,More than 10 times a week,NaN
1488,952421.0,NaN,952421_2024-11-19,NaN,5.0,1,3,1,Agree,Disagree,...,51.33902,NaN,40.99799,NaN,No Risk,NaN,NaN,More than 10 times a week,More than 10 times a week,NaN
1489,981476.0,NaN,981476_2024-11-01,NaN,2.0,0,1,1,Strongly Agree,Agree,...,59.58992,NaN,37.71737,NaN,No Risk,NaN,NaN,More than 10 times a week,More than 10 times a week,NaN
1490,983777.0,NaN,983777_2024-11-22,NaN,2.0,1,0,1,Agree,Strongly Agree,...,59.76309,NaN,39.42751,NaN,No Risk,NaN,NaN,More than 10 times a week,5-10 times a week,NaN


In [1054]:
transcripts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   New ID                  1375 non-null   float64
 1   Old ID                  1470 non-null   float64
 2   Recording File Name     1492 non-null   object 
 3   Text                    1470 non-null   object 
 4   CEL Total               1492 non-null   float64
 5   CELVAL1                 1492 non-null   int64  
 6   CELVAL2                 1492 non-null   int64  
 7   CELVAL3                 1492 non-null   int64  
 8   CEL1                    1492 non-null   object 
 9   CEL2                    1492 non-null   object 
 10  CEL3                    1492 non-null   object 
 11  SQ                      1470 non-null   float64
 12  {E} Engagement          1491 non-null   float64
 13  {R} Reversed CEL        1492 non-null   int64  
 14  Personality Raw         1485 non-null   

In [1055]:
transcripts_df['New ID'] = pd.to_numeric(transcripts_df['New ID'], errors='coerce') \
                           .replace(0, np.nan) \
                           .astype('Int64') \
                           .astype(str)

transcripts_df['Old ID'] = pd.to_numeric(transcripts_df['Old ID'], errors='coerce') \
                           .replace(0, np.nan) \
                           .astype('Int64') \
                           .astype(str)

In [1056]:
transcripts_df['New ID'] = transcripts_df['New ID'].astype(str)  # Ensure consistency
old_id_mapping = {str(k): v for k, v in old_id_mapping.items()}  # Convert dictionary keys to string
# Replace values in 'Old ID' column based on 'New ID'
transcripts_df['Old ID'] = transcripts_df['New ID'].map(old_id_mapping).fillna(transcripts_df['Old ID'])
transcripts_df["New ID"] = transcripts_df["New ID"].replace("<NA>", np.nan)
transcripts_df["Old ID"] = transcripts_df["Old ID"].replace("<NA>", np.nan)

In [1057]:
transcripts_df['New ID']

0          NaN
1       938730
2       938936
3          NaN
4       938879
         ...  
1487    941934
1488    952421
1489    981476
1490    983777
1491    997606
Name: New ID, Length: 1492, dtype: object

In [1058]:
transcripts_df['Old ID'] = transcripts_df['Old ID'].astype(int)
transcripts_df['Old ID'] = transcripts_df['Old ID'].astype(str)

In [ ]:
# First merge on New ID
metadata_df = pd.merge(transcripts_df, demographics_df, left_on='New ID', right_on='New ID', how='left', suffixes=('', '_demographics'))
metadata_df
# Find rows where the first merge failed (Age column is NaN)
missing_merge = metadata_df[metadata_df['Age'].isna()]
print(len(missing_merge))
# Perform second merge for rows with missing matches, using Old ID
fallback_merge = pd.merge(
    missing_merge[['Old ID', 'New ID']],  # Use only relevant columns
    demographics_df, 
    left_on='Old ID', 
    right_on='Old ID',  # Match Old ID to Old ID in demographics_df
    how='left'
)

# Combine the fallback results with the primary merge
metadata_df.update(fallback_merge)
# Remove unnecessary or duplicate columns
metadata_df.drop(columns=['New ID_demographics', 'Old ID_demographics','KlaatchID'], inplace=True, errors='ignore')
metadata_df.rename(columns={'Recording File Name': 'Filename'}, inplace=True)
metadata_df.rename(columns={'Old ID': 'KlaatchID'}, inplace=True)
metadata_df['Filename'] = metadata_df['Filename'].str.lower().str.strip()
#metadata_df['KlaatchID'] = metadata_df['Filename'].apply(lambda x: x.split('_')[0])
# Display the final dataframe
metadata_df


1492


,New ID,KlaatchID,Filename,Text,CEL Total,CELVAL1,CELVAL2,CELVAL3,CEL1,CEL2,...,Ethnicity,Race,PartnershipStatus,MedicareStatus,VeteranStatus,Education,Living_With,Monthly Income,Cognitive Impaired,Dementia
0,NaN,559,559_2021-01-22,"Hello. OK. Sure. Yesterday, I went to an appoi...",5.0,1,0,4,Agree,Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,938730,66,66_2021-01-26,Hello. Please leave a message after the tone. ...,3.0,1,1,1,Agree,Agree,...,Not Hispanic or Latino,White,Single,Yes,No,High school diploma,Self Only,1841.96,NaN,YES
2,938936,340,340_2021-01-26,"Yes. I'm fine, how are you? Ok. Well, what I u...",2.0,1,0,1,Agree,Strongly Agree,...,Not Hispanic or Latino,Black or African American,Widowed,No,No,High school diploma,Self Only,1630.00,NaN,NaN
3,NaN,343,343_2021-01-26,"Hello. All right. I'll do it now. Sure. No, no...",1.0,1,0,0,Agree,Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,938879,383,383_2021-01-27,"Hello. Hi, I'm. Ok. OK. Yes, okay. I wake up, ...",5.0,1,3,1,Agree,Disagree,...,Hispanic or Latino,White,Separated,Yes,Unknown,Bachelor's degree,Self Only,885.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,941934,810,941934_2024-11-07,NaN,1.0,0,0,1,Strongly Agree,Strongly Agree,...,Not Hispanic or Latino,White,Widowed,Yes,No,Master's degree,Self Only,5234.90,NaN,NaN
1488,952421,818,952421_2024-11-19,NaN,5.0,1,3,1,Agree,Disagree,...,Not Hispanic or Latino,White,Unknown,Yes,No,Unknown,Self Only,2357.00,NaN,NaN
1489,981476,852,981476_2024-11-01,NaN,2.0,0,1,1,Strongly Agree,Agree,...,Not Hispanic or Latino,White,Unknown,No,No,Unknown,Self Only,2382.00,NaN,NaN
1490,983777,854,983777_2024-11-22,NaN,2.0,1,0,1,Agree,Strongly Agree,...,Not Hispanic or Latino,Black or African American,Married,Yes,No,High school diploma,Spouse,1919.50,NaN,NaN


In [1061]:
# Define the root folder where the files are stored
root_folder = '/sandata/karthik9/nwa'

file_data = []  # List to store information about each file
incorrect_files = []
mp3_count = 0

# Regex pattern for valid filenames: "sample_YYYY-MM-DD"
valid_filename_pattern = re.compile(r'^[0-9]+_\d{4}-\d{2}-\d{2}$')

# Traverse through subfolders
for subfolder in os.listdir(root_folder):
    subfolder_path = os.path.join(root_folder, subfolder)

    # Check if the subfolder path is actually a directory
    if os.path.isdir(subfolder_path):
        for file_name in os.listdir(subfolder_path):
            if file_name.endswith(".mp3"):  # Assuming the audio files are in .mp3 format
                file_path = os.path.join(subfolder_path, file_name)
                mp3_count +=1
                # Clean the filename
                cleaned_name = file_name.lstrip('T').rsplit('_', 1)[0]
                cleaned_name = cleaned_name.replace('R_The_towers_', '').replace('__', '_').replace('wb', '').strip('_')
                # Check if the cleaned name is already in the correct format (YYYY-MM-DD)
                if valid_filename_pattern.match(cleaned_name):
                    # If it's already in the correct format, use it as is
                    new_filename = cleaned_name
                else:
                    # Split the cleaned name to identify the date part
                    parts = cleaned_name.split('_')
                    if len(parts) > 1:
                        date_parts = parts[1].split('-')
                        if len(date_parts) == 3 and all([p.isdigit() for p in date_parts]):
                            # Add leading zeros to the month and day if needed
                            year = date_parts[2]
                            month = date_parts[0].zfill(2)  # Add leading zero to month if necessary
                            day = date_parts[1].zfill(2)    # Add leading zero to day if necessary

                            # Reorder the date to YYYY-MM-DD format
                            new_date = f"{year}-{month}-{day}"
                            new_filename = f"{parts[0]}_{new_date}"
                            klaatchid = parts[0]
                            date = new_date
                            
                        else:
                            new_filename = cleaned_name  # Handle incorrect date format
                            klaatchid = None
                            date = None
                    else:
                        new_filename = cleaned_name  # Handle missing underscores
                        klaatchid = None
                        date = None

                # Append the correct filename and file path
                file_data.append({
                    'Filename': new_filename,
                    'Filepath': file_path,
                    'Cleaned Name': cleaned_name,
                    'KlaatchID': klaatchid,
                    'Date':date,
                    'Original_Filename':file_name
                })

                # Check if the new filename matches the desired format
                if not valid_filename_pattern.match(new_filename):
                    # If it doesn't match, add it to the incorrect_files list
                    incorrect_files.append(new_filename)

print(mp3_count)


1499


In [1062]:
# Convert the file data into a DataFrame for further processing
df_audio = pd.DataFrame(file_data)
df_audio['Filename'] = df_audio['Filename'].str.lower().str.strip()
df_audio['Cleaned Name'] = df_audio['Cleaned Name'].str.lower().str.strip()

In [1063]:
print(df_audio.shape)
df_audio[['Filename','KlaatchID','Date']]

(1499, 6)


,Filename,KlaatchID,Date
0,753_2023-04-20,753,2023-04-20
1,32_2024-09-20,32,2024-09-20
2,485_2021-04-29,485,2021-04-29
3,718_2024-01-03,718,2024-01-03
4,494_2021-06-03,494,2021-06-03
...,...,...,...
1494,943780_2024-12-09,943780,2024-12-09
1495,999746_2024-12-03,999746,2024-12-03
1496,279_2023-06-26,279,2023-06-26
1497,651_20221-12-19,651,20221-12-19


In [1064]:
metadata_df['Date']= metadata_df['Filename'].apply(lambda x: x.split('_')[1])
metadata_df


,New ID,KlaatchID,Filename,Text,CEL Total,CELVAL1,CELVAL2,CELVAL3,CEL1,CEL2,...,Race,PartnershipStatus,MedicareStatus,VeteranStatus,Education,Living_With,Monthly Income,Cognitive Impaired,Dementia,Date
0,NaN,559,559_2021-01-22,"Hello. OK. Sure. Yesterday, I went to an appoi...",5.0,1,0,4,Agree,Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-22
1,938730,66,66_2021-01-26,Hello. Please leave a message after the tone. ...,3.0,1,1,1,Agree,Agree,...,White,Single,Yes,No,High school diploma,Self Only,1841.96,NaN,YES,2021-01-26
2,938936,340,340_2021-01-26,"Yes. I'm fine, how are you? Ok. Well, what I u...",2.0,1,0,1,Agree,Strongly Agree,...,Black or African American,Widowed,No,No,High school diploma,Self Only,1630.00,NaN,NaN,2021-01-26
3,NaN,343,343_2021-01-26,"Hello. All right. I'll do it now. Sure. No, no...",1.0,1,0,0,Agree,Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-26
4,938879,383,383_2021-01-27,"Hello. Hi, I'm. Ok. OK. Yes, okay. I wake up, ...",5.0,1,3,1,Agree,Disagree,...,White,Separated,Yes,Unknown,Bachelor's degree,Self Only,885.00,NaN,NaN,2021-01-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,941934,810,941934_2024-11-07,NaN,1.0,0,0,1,Strongly Agree,Strongly Agree,...,White,Widowed,Yes,No,Master's degree,Self Only,5234.90,NaN,NaN,2024-11-07
1488,952421,818,952421_2024-11-19,NaN,5.0,1,3,1,Agree,Disagree,...,White,Unknown,Yes,No,Unknown,Self Only,2357.00,NaN,NaN,2024-11-19
1489,981476,852,981476_2024-11-01,NaN,2.0,0,1,1,Strongly Agree,Agree,...,White,Unknown,No,No,Unknown,Self Only,2382.00,NaN,NaN,2024-11-01
1490,983777,854,983777_2024-11-22,NaN,2.0,1,0,1,Agree,Strongly Agree,...,Black or African American,Married,Yes,No,High school diploma,Spouse,1919.50,NaN,NaN,2024-11-22


In [1065]:
# # Drop duplicates based on 'Filename' column
df_audio = df_audio.drop_duplicates(subset='Filename', keep='first')
metadata_df = metadata_df.drop_duplicates(subset='Filename', keep='first')
print(df_audio.shape)
print(metadata_df.shape)

(1493, 6)
(1492, 42)


In [1066]:
# Perform the merge based on the 'Filename' column
merged_df = pd.merge(metadata_df, df_audio, on='Filename')

merged_df.drop(columns=['KlaatchID_y', 'Date_y'], inplace=True, errors='ignore')
merged_df.rename(columns={'KlaatchID_x': 'KlaatchID'}, inplace=True)
merged_df.rename(columns={'Date_x': 'Date'}, inplace=True)

# Print the merged DataFrame shape
print("Merged DataFrame shape:", merged_df.shape)

Merged DataFrame shape: (1473, 45)


In [1067]:
# Find filenames in metadata_df that are not in df_audio
unmatched_metadata_filenames = metadata_df[~metadata_df['Filename'].isin(df_audio['Filename'])]['Filename'].tolist()

# Find filenames in df_audio that are not in metadata_df
unmatched_audio_filenames = df_audio[~df_audio['Filename'].isin(metadata_df['Filename'])]['Filename'].tolist()

no_demographics_ids = merged_df.loc[merged_df[['Age', 'Gender', 'Race']].isna().all(axis=1), 'KlaatchID'].drop_duplicates().tolist()
#no_demographics_ids = sorted(no_demographics_ids)


print(len(no_demographics_ids))
print("No demographics in Merged Data:", no_demographics_ids)

print(len(unmatched_metadata_filenames))
print("Unmatched Filenames in metadata_df:", unmatched_metadata_filenames)

print(len(unmatched_audio_filenames))
print("Unmatched Filenames in df_audio:", unmatched_audio_filenames)


59
No demographics in Merged Data: ['559', '343', '50', '617', '629', '635', '241', '466', '268', '332', '642', '220', '483', '255', '577', '534', '372', '693', '271', '673', '180', '458', '56', '479', '485', '536', '611', '533', '644', '61', '653', '641', '217', '494', '264', '599', '267', '190', '613', '607', '685', '561', '682', '72', '709', '103', '106', '108', '113', '114', '128', '132', '138', '143', '146', '148', '154', '155', '156']
19
Unmatched Filenames in metadata_df: ['319_2022-02-03', '522_12-10-21', '103_2021-11-22', '103_2022-02-14', '103_2022-07-04', '105_2021-06-10', '105_2021-07-15', '106_2021-07-29', '107_2021-07-17', '113_2021-07-28', '114_2021-07-12', '117_2021-07-18', '121_2021-07-18', '124_2021-06-19', '128_2021-07-28', '138_2021-06-23', '144_2021-07-12', '155_2021-07-21', '156_2021-07-20']
20
Unmatched Filenames in df_audio: ['131_2021-06-04', '131_2021-04-21', '672_2020-12-30', '1007_2021-03-10', '820_2023-07-31', '698_2021-12-27', '459_2024-07-29', '489_2023-0

In [1068]:
new_transcripts = pd.read_csv('/home/karthik9/new_transcripts.csv')
# Step 2: Merge filtered_df with new_transcripts on the relevant columns
merged_df = merged_df.merge(
    new_transcripts[['Filename', 'Filepath', 'Original_Filename', 'Text']],
    on=['Filename'],
    how='left',
    suffixes=('', '_new')
)
merged_df

# Update 'Text' column where 'Text' is 'NOT FOUND' or NaN
merged_df.loc[(merged_df['Text'] == 'NOT FOUND') | (merged_df['Text'].isna()), 'Text'] = merged_df['Text_new']
merged_df.drop(columns=['Filepath_new', 'Original_Filename_new', 'Text_new'], inplace=True)

merged_df


,New ID,KlaatchID,Filename,Text,CEL Total,CELVAL1,CELVAL2,CELVAL3,CEL1,CEL2,...,VeteranStatus,Education,Living_With,Monthly Income,Cognitive Impaired,Dementia,Date,Filepath,Cleaned Name,Original_Filename
0,NaN,559,559_2021-01-22,"Hello. OK. Sure. Yesterday, I went to an appoi...",5.0,1,0,4,Agree,Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-22,/sandata/karthik9/nwa/New_audio/559_01-22-2021...,559_01-22-2021_11-40am,559_01-22-2021_11-40am_wb.mp3
1,938730,66,66_2021-01-26,Hello. Please leave a message after the tone. ...,3.0,1,1,1,Agree,Agree,...,No,High school diploma,Self Only,1841.96,NaN,YES,2021-01-26,/sandata/karthik9/nwa/New_audio/66_01-26-2021_...,66_01-26-2021_15-20min,66_01-26-2021_15-20min_wb.mp3
2,938936,340,340_2021-01-26,"Yes. I'm fine, how are you? Ok. Well, what I u...",2.0,1,0,1,Agree,Strongly Agree,...,No,High school diploma,Self Only,1630.00,NaN,NaN,2021-01-26,/sandata/karthik9/nwa/New_audio/340_01-26-2021...,340_01-26-2021_15-05min,340_01-26-2021_15-05min_wb.mp3
3,NaN,343,343_2021-01-26,"Hello. All right. I'll do it now. Sure. No, no...",1.0,1,0,0,Agree,Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-26,/sandata/karthik9/nwa/New_audio/343_01-26-2021...,343_01-26-2021_08-43min,343_01-26-2021_08-43min_wb.mp3
4,938879,383,383_2021-01-27,"Hello. Hi, I'm. Ok. OK. Yes, okay. I wake up, ...",5.0,1,3,1,Agree,Disagree,...,Unknown,Bachelor's degree,Self Only,885.00,NaN,NaN,2021-01-27,/sandata/karthik9/nwa/New_audio/383_01-27-2021...,383_01-27-2021_15-35min,383_01-27-2021_15-35min_wb.mp3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,941934,810,941934_2024-11-07,Okay. Okay. All right. Okay. Yes. Oh. Okay. O...,1.0,0,0,1,Strongly Agree,Strongly Agree,...,No,Master's degree,Self Only,5234.90,NaN,NaN,2024-11-07,/sandata/karthik9/nwa/New_audio/R_The_towers_9...,941934_11-7-2024_8-54am (1),R_The_towers_941934_11-7-2024_8-54am (1)_wb.mp3
1469,952421,818,952421_2024-11-19,"Hello? Hello? Yes. Lorna, how are you? Good. ...",5.0,1,3,1,Agree,Disagree,...,No,Unknown,Self Only,2357.00,NaN,NaN,2024-11-19,/sandata/karthik9/nwa/New_audio/R_The_towers_9...,952421_11-19-2024_10-21am,R_The_towers_952421_11-19-2024_10-21am_wb.mp3
1470,981476,852,981476_2024-11-01,"Oh, I didn't know. Very good. Oh, no, no prob...",2.0,0,1,1,Strongly Agree,Agree,...,No,Unknown,Self Only,2382.00,NaN,NaN,2024-11-01,/sandata/karthik9/nwa/New_audio/R_The_towers_9...,981476_11-1-2024_4-40pm (1),R_The_towers_981476_11-1-2024_4-40pm (1)_wb.mp3
1471,983777,854,983777_2024-11-22,"Hello? Yes? Hi, how are you? Pretty good. I'm...",2.0,1,0,1,Agree,Strongly Agree,...,No,High school diploma,Spouse,1919.50,NaN,NaN,2024-11-22,/sandata/karthik9/nwa/New_audio/R_The_towers_9...,983777_11-22-2024_4-31pm,R_The_towers_983777_11-22-2024_4-31pm_wb.mp3


In [1069]:
filtered_df = merged_df[merged_df["New ID"].isna()][["New ID", "KlaatchID", "Filename", "Age"]]
filtered_df


,New ID,KlaatchID,Filename,Age
0,NaN,559,559_2021-01-22,NaN
3,NaN,343,343_2021-01-26,NaN
6,NaN,50,50_2021-01-29,NaN
18,NaN,617,617_2021-02-05,NaN
19,NaN,629,629_2021-02-05,NaN
...,...,...,...,...
1446,NaN,154,154_2021-04-27,NaN
1447,NaN,155,155_2021-04-20,NaN
1448,NaN,155,155_2021-06-02,NaN
1449,NaN,156,156_2021-05-05,NaN


In [1070]:
# no_transcripts = merged_df.loc[(merged_df['Text'] == 'NOT FOUND') | (merged_df['Text'].isna()), ['Filename','Filepath', 'Original_Filename','Text']]
# no_transcripts


In [1071]:
stopwords_merged_df = merged_df

In [1072]:
import pandas as pd
import re

# Assuming merged_df is already defined as per your initial code

# Define the patterns to remove
patterns_to_remove = [
    'lisa\w*',
    'lisa',
    'speaker1','speaker2'
    'strongly agree',
    'strongly disagree',
    'strongly_agree',
    'strongly_disagree',
    'agree\w*',          # Matches agree, agreed, agreement, agreeable, agreeing
    'disagree\w*',       # Matches disagree, disagreed, disagreement, disagreeing
    'neutral',
    'Speaker',
    'strongly agree\w*', # Matches strongly agree variations
    'strongly disagree\w*',
    'strongly_agree\w*',
    'strongly_disagree\w*',
    'Speaker\w*',
    'nn\w*',
    'nnspeaker\w*',
    '!', '\$', ',', '\.', '\.\.', '\.\s*mm hmm', '\.\.',
    '0', '00 \]', '0:00', '0:00:00',
    '1', '10', '100', '11', '12', '13', '14', '15', '16', '18',
    '2', '20', '20 years', '25',
    '3', '30', '4', '40',
    '5', '5 to 10', '50', '90',
    ':', ': 00 \]', '\?', '\]', '\] hello', '\] hello \.',
    "don't\w*", 'not\w*', 'strongly\w*', 'definitely\w*', 'neither\w*'
]



# Regex pattern to identify timestamps like [00:24:00]
timestamp_pattern = r'\[\d{2}:\d{2}:\d{2}\]'

# Note: Do NOT escape patterns that contain regex operators like \w
escaped_patterns = []
for pattern in patterns_to_remove:
    if '\\w' in pattern:
        escaped_patterns.append(pattern)  # Keep regex operators
    else:
        escaped_patterns.append(re.escape(pattern))  # Escape others

words_regex = r'\b(?:' + '|'.join(escaped_patterns) + r')\b'


# Combine word/phrase patterns with timestamp pattern
full_regex_pattern = f'(?:{words_regex})|(?:{timestamp_pattern})'

# Remove the patterns from the 'Text' column
stopwords_merged_df['Text'] =  stopwords_merged_df['Text'].str.replace(
    full_regex_pattern, 
    '', 
    regex=True, 
    flags=re.IGNORECASE
)

# Clean up any extra whitespace
stopwords_merged_df['Text'] = stopwords_merged_df['Text'].str.replace(r'\s+', ' ', regex=True).str.strip()

# Verify the changes
#print(stopwords_merged_df[['Filename', 'Text']].head())


In [1073]:
stopwords_merged_df.to_csv('zzzdata.csv', index=False)
# stopwords_merged_df1 = stopwords_merged_df[['Filename','Text','CEL Total']]
# stopwords_merged_df1.to_csv('zdata.csv')

In [737]:
stopwords_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1473 entries, 0 to 1472
Data columns (total 45 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   New ID                  1473 non-null   object 
 1   KlaatchID               1473 non-null   object 
 2   Filename                1473 non-null   object 
 3   Text                    1473 non-null   object 
 4   CEL Total               1473 non-null   float64
 5   CELVAL1                 1473 non-null   int64  
 6   CELVAL2                 1473 non-null   int64  
 7   CELVAL3                 1473 non-null   int64  
 8   CEL1                    1473 non-null   object 
 9   CEL2                    1473 non-null   object 
 10  CEL3                    1473 non-null   object 
 11  SQ                      1451 non-null   float64
 12  {E} Engagement          1472 non-null   float64
 13  {R} Reversed CEL        1473 non-null   int64  
 14  Personality Raw         1468 non-null   

In [736]:
def insert_data(df, table_name):
    """
    Inserts data into the specified table.

    Args:
    df (pd.DataFrame): The DataFrame containing the data to insert.
    table_name (str): The name of the database table to insert data into.
    """
    # Establish a connection to the database
    connection = connect_to_db()
    cursor = connection.cursor()

    # Construct the INSERT query dynamically based on the table name
    insert_query = f"""
    INSERT IGNORE INTO {table_name} (
        message_id, message, KlaatchID, Date, CEL_Total, CELVAL1, CELVAL2, CELVAL3, 
        Age
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Convert DataFrame to a list of tuples, including all required columns
    data = df[['Filename', 'Text', 'KlaatchID', 'Date', 'CEL Total', 
               'CELVAL1', 'CELVAL2', 'CELVAL3', 'Age']].values.tolist()
    
    try:
        cursor.executemany(insert_query, data)
        connection.commit()
        print(f"Inserted {cursor.rowcount} rows successfully into table {table_name}.")
    except mysql.connector.Error as err:
        print(f"Error: {err}")
    finally:
        cursor.close()
        connection.close()

# Example usage
#insert_data(stopwords_merged_df, "merged_data")


In [36]:
# Group by KlaatchID and calculate mean, std, and count
grouped_stats = merged_df.groupby('KlaatchID')['CEL Total'].agg(['mean', 'std', 'count'])

# Reset index if needed
grouped_stats = grouped_stats.reset_index()

# Filter out rows where 'std' is NaN
filtered_stats = grouped_stats[grouped_stats['std'].notna()]

# Sort by 'std' in descending order
sorted_stats = filtered_stats.sort_values(by='count', ascending=False)

# Display sorted DataFrame
sorted_stats

# Count how many groups have std == 0.000000
#zero_std_count = (grouped_stats['std'] == 0.000000).sum()

# Display the count
#print(f"Number of groups with std = 0.000000: {zero_std_count}")


,KlaatchID,mean,std,count
16,144,3.923077,2.564551,13
74,45,3.916667,1.083625,12
143,606,2.916667,1.378954,12
306,86,0.636364,1.026911,11
41,265,1.454545,1.572491,11
...,...,...,...,...
122,56,2.0,1.414214,2
121,559,5.0,0.000000,2
258,786,2.5,0.707107,2
259,787,2.0,1.414214,2


In [37]:
kk =merged_df[merged_df['KlaatchID'] == '45']
kk[['KlaatchID','Filename','New ID' ]]

,KlaatchID,Filename,New ID
16,45,45_2021-02-05,938945
186,45,45_2021-09-20,938945
222,45,45_2021-11-16,938945
329,45,45_2022-02-09,938945
501,45,45_2022-06-13,938945
617,45,45_2022-09-06,938945
727,45,45_2023-01-09,938945
880,45,45_2023-05-02,938945
1024,45,45_2023-09-13,938945
1144,45,45_2024-01-08,938945


In [39]:
# Count of records in each DataFrame
print("Total records in metadata_df:", metadata_df['Filename'].count())
print("Total records in df_audio:", df_audio['Filename'].count())
print("Total records in merged_df:", merged_df['Filename'].count())

# Count unique filenames
print("Unique filenames in metadata_df:", metadata_df['Filename'].nunique())
print("Unique filenames in df_audio:", df_audio['Filename'].nunique())
print("Unique filenames in merged_df:", merged_df['Filename'].nunique())


Total records in metadata_df: 1493
Total records in df_audio: 1496
Total records in merged_df: 1473
Unique filenames in metadata_df: 1493
Unique filenames in df_audio: 1496
Unique filenames in merged_df: 1473


In [40]:
# Function to convert to WAV if necessary
def convert_to_wav(audio_file_path):
    if not audio_file_path.endswith('.wav'):
        print(f"Converting {audio_file_path} to WAV format...")
        audio = AudioSegment.from_file(audio_file_path)
        new_file_path = os.path.splitext(audio_file_path)[0] + ".wav"
        audio.export(new_file_path, format="wav")
        print(f"Conversion complete: {new_file_path}")
        return new_file_path
    return audio_file_path
    

# Function to preprocess the audio (resampling if necessary)
def preprocess_audio(audio_file_path, target_sample_rate=16000):
    audio_file_path = convert_to_wav(audio_file_path)
    waveform, sampling_rate = torchaudio.load(audio_file_path)
    if sampling_rate != target_sample_rate:
        resample_transform = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=target_sample_rate)
        waveform = resample_transform(waveform)
    return waveform, target_sample_rate

In [70]:
# Directory where Whisper feature files will be saved
WHISPER_FEATURES_DIR = '/sandata/karthik9/whisper_new'

# Function to check if a Whisper file is already processed
def is_whisper_file_processed(filename):
    base_name = filename.rsplit('.mp3', 1)[0]  # Removes only the last occurrence of '.mp3'
    whisper_features_file = os.path.join(WHISPER_FEATURES_DIR, f"{base_name}_whisper_features.npy")
    
    return os.path.exists(whisper_features_file), whisper_features_file

# Function to save Whisper extracted features to a file
def save_whisper_features_to_file(features, filename):
    base_name = filename.rsplit('.mp3', 1)[0]
    # Ensure the directory exists
    if not os.path.exists(WHISPER_FEATURES_DIR):
        os.makedirs(WHISPER_FEATURES_DIR)
    
    # Create the full file path
    features_file_path = os.path.join(WHISPER_FEATURES_DIR, f"{base_name}_whisper_features.npy")
    
    # Save the features as a NumPy file
    np.save(features_file_path, features)
    
    return features_file_path

# Extract Whisper features
def extract_whisper_features(waveform, sample_rate):
   
    print("Loading Whisper processor and model...")
    processor = WhisperProcessor.from_pretrained('openai/whisper-base')
    model = WhisperModel.from_pretrained('openai/whisper-base')
    model.eval()  # Set the model to evaluation mode

    print("Processing audio and preparing input tensors for Whisper...")
    input_features = processor(waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt").input_features

    print("Extracting Whisper features...")
    with torch.no_grad():
        encoder_outputs = model.get_encoder()(input_features)  # Extract encoder features
        features = encoder_outputs.last_hidden_state  # Shape: (batch_size, sequence_length, hidden_size)

    print(f"Whisper features extracted. Shape: {features.shape}")

    # Convert features to NumPy array
    features_np = features.squeeze().numpy()  # Remove batch dimension if batch_size=1
    return features_np

# Process the dataset and insert Whisper file paths into the database
def process_and_store_whisper_features(merged_df):
    connection = connect_to_db()
    processed_files = []

    for index, row in merged_df.iterrows():
        audio_file_path = row['Filepath']
        filename = row['Filename']
        Original_Filename = row['Original_Filename']
        
        # Check if the file is already processed in the Whisper table
        is_processed, features_file_path = is_whisper_file_processed(Original_Filename)
        
        if is_processed:
            processed_files.append({"Filename": filename, "features_file_path": features_file_path})
            continue  
        
        try:
            print(filename,Original_Filename)
            #Preprocess and extract Whisper features
            waveform, sample_rate = preprocess_audio(audio_file_path)
            whisper_features = extract_whisper_features(waveform, sample_rate)
            
            # Print the feature vector size (shape)
            print(f"Feature vector size for {Original_Filename}: {whisper_features.shape}")
            
            # Save the extracted Whisper features to a file
            whisper_features_file_path = save_whisper_features_to_file(whisper_features, Original_Filename)
            
            # Store the Whisper file path in the database
            #insert_whisper_file_path_into_db(connection, filename, whisper_features_file_path)
            print(f"Whisper features for {filename} stored successfully in {whisper_features_file_path}.")

        except Exception as e:
            print(f"Error processing {audio_file_path} with Whisper: {e}")

    processed_df = pd.DataFrame(processed_files, columns=["Filename", "features_file_path"])
    connection.close()
    return processed_df

# Example usage
whisper_processed_df = process_and_store_whisper_features(merged_df)


In [71]:
whisper_processed_df

,Filename,features_file_path
0,559_2021-01-22,/sandata/karthik9/whisper_new/559_01-22-2021_1...
1,66_2021-01-26,/sandata/karthik9/whisper_new/66_01-26-2021_15...
2,340_2021-01-26,/sandata/karthik9/whisper_new/340_01-26-2021_1...
3,343_2021-01-26,/sandata/karthik9/whisper_new/343_01-26-2021_0...
4,383_2021-01-27,/sandata/karthik9/whisper_new/383_01-27-2021_1...
...,...,...
1468,941934_2024-11-07,/sandata/karthik9/whisper_new/R_The_towers_941...
1469,952421_2024-11-19,/sandata/karthik9/whisper_new/R_The_towers_952...
1470,981476_2024-11-01,/sandata/karthik9/whisper_new/R_The_towers_981...
1471,983777_2024-11-22,/sandata/karthik9/whisper_new/R_The_towers_983...


In [47]:
def insert_feature_table_into_db(connection, dataframe, table_name):
    
    cursor = connection.cursor()
    
    # Generate the column placeholders for the query
    columns = ', '.join(dataframe.columns)
    placeholders = ', '.join(['%s'] * len(dataframe.columns))
    
    # Prepare the SQL query for insertion
    query = f"""
    INSERT INTO {table_name} ({columns})
    VALUES ({placeholders})
    """
    
    # Convert the DataFrame rows to a list of tuples
    records = list(dataframe.itertuples(index=False, name=None))
    
    # Insert the records into the database
    cursor.executemany(query, records)
    connection.commit()
    print(f"{cursor.rowcount} rows inserted into {table_name}.")
    cursor.close()


In [73]:
def compute_mean_median(file_path):
    """
    Computes the per-dimension mean and median for a given .npy file.

    Parameters:
        file_path (str): Path to the .npy file.

    Returns:
        tuple: Two numpy arrays containing the mean and median for each dimension.
    """
    try:
        # Ensure the file exists
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")

        # Load the NumPy array
        array = np.load(file_path)

        # Determine the axes to compute over (all except the last axis)
        if array.ndim == 2:
            # Shape: (1500, 512)
            axes = 0
        elif array.ndim == 3:
            # Shape: (2, 1500, 512)
            axes = (0, 1)
        else:
            raise ValueError(f"Unsupported array dimensions: {array.shape}")

        # Compute mean and median along the specified axes
        mean_vals = np.mean(array, axis=axes)
        median_vals = np.median(array, axis=axes)

        # Validate the result shapes
        if mean_vals.shape != (512,) or median_vals.shape != (512,):
            raise ValueError(f"Unexpected result shape: mean={mean_vals.shape}, median={median_vals.shape}")

        return mean_vals, median_vals

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        # Return arrays of NaNs if there's an error
        return np.full(512, np.nan), np.full(512, np.nan)

# Lists to store the mean and median arrays
mean_list = []
median_list = []

# Iterate over each file path and compute mean and median
for idx, row in whisper_processed_df.iterrows():
    file_path = row['features_file_path']
    
    # Compute mean and median
    mean_vals, median_vals = compute_mean_median(file_path)
    
    # Append to the respective lists
    mean_list.append(mean_vals)
    median_list.append(median_vals)

# Convert the lists to DataFrames
mean_df = pd.DataFrame(mean_list, columns=[f'dim_{i}' for i in range(512)])
median_df = pd.DataFrame(median_list, columns=[f'dim_{i}' for i in range(512)])

# Optionally, include the file paths or identifiers
mean_df.insert(0, 'group_id', whisper_processed_df['Filename'])
median_df.insert(0, 'group_id', whisper_processed_df['Filename'])


In [74]:
median_df 

,group_id,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,...,dim_502,dim_503,dim_504,dim_505,dim_506,dim_507,dim_508,dim_509,dim_510,dim_511
0,559_2021-01-22,0.263402,-0.109678,0.164839,-1.022715,0.547466,1.534519,-0.266129,-0.206032,0.209028,...,0.085625,-0.471373,-0.744186,-0.020223,0.354562,0.079967,0.149645,-0.353890,0.209563,0.369187
1,66_2021-01-26,-0.023211,-0.234569,0.173479,-0.619449,0.509412,0.574127,-0.248781,-0.115399,-0.122104,...,0.360241,-0.299386,-0.273263,0.031982,0.329269,0.214321,0.308786,-0.148437,-0.032440,0.160715
2,340_2021-01-26,0.031608,-0.016620,0.460649,-0.901895,0.701194,1.267238,-0.132400,-0.283320,0.093290,...,0.349277,-0.806896,-0.833856,-0.046660,0.428650,0.123274,0.068465,-0.434517,0.176311,0.475421
3,343_2021-01-26,-0.030930,-0.299418,0.190314,-0.574210,0.568031,0.503544,-0.154806,-0.145413,-0.134015,...,0.440828,-0.245475,-0.133994,0.115500,0.223310,0.001337,0.393475,-0.228038,-0.143641,0.235286
4,383_2021-01-27,-0.028178,-0.127283,0.252597,-0.861443,0.857720,0.870966,-0.211499,-0.231921,-0.022566,...,0.309479,-0.258111,-0.528164,-0.145514,0.272675,0.189025,0.327568,-0.111582,-0.017942,0.235279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,941934_2024-11-07,-0.049736,-0.111479,0.121401,-0.623412,0.648028,0.602230,-0.007144,-0.205515,-0.507898,...,0.303787,-0.229650,-0.044096,-0.172942,0.067588,0.013118,0.313039,-0.037251,0.091475,0.208970
1469,952421_2024-11-19,-0.074439,-0.056408,0.123652,-0.277473,0.182573,0.312414,0.159376,-0.371311,-0.317633,...,0.115751,-0.073866,0.059007,-0.430159,0.067964,0.026781,0.308927,0.080771,-0.189482,0.088772
1470,981476_2024-11-01,-0.004254,-0.119211,0.087904,-0.486616,0.649047,0.527288,-0.130802,-0.429370,-0.330180,...,0.315757,-0.102892,-0.154528,-0.176320,-0.058409,0.104448,0.388173,0.011822,-0.061125,0.309502
1471,983777_2024-11-22,-0.130247,-0.183568,-0.139036,-0.265875,0.459765,0.363883,-0.009049,-0.477304,-0.395824,...,0.340502,-0.020614,-0.084908,-0.108437,-0.156296,-0.166919,0.294220,-0.070403,-0.186831,0.336732


In [75]:
melted_df = mean_df.melt(
    id_vars=["group_id"],  # Columns to keep as is
    var_name="feat",            # Name for the feature column
    value_name="value"          # Name for the value column
)

# Optionally, calculate `group_norm` (if needed)
melted_df["group_norm"] = melted_df["value"]


melted_df1 = median_df.melt(
    id_vars=["group_id"],  # Columns to keep as is
    var_name="feat",            # Name for the feature column
    value_name="value"          # Name for the value column
)

# Optionally, calculate `group_norm` (if needed)
melted_df1["group_norm"] = melted_df1["value"]


In [76]:
connection = connect_to_db()

# Insert the DataFrame into the database
insert_feature_table_into_db(connection, melted_df, 'feat$whisper_mean_n$merged_data$message_id')

# Close the connection
connection.close()

754176 rows inserted into feat$whisper_mean_n$merged_data$message_id.


In [77]:
connection = connect_to_db()

# Insert the DataFrame into the database
insert_feature_table_into_db(connection, melted_df1, 'feat$whisper_median_n$merged_data$message_id')

# Close the connection
connection.close()

754176 rows inserted into feat$whisper_median_n$merged_data$message_id.


OPEN SMILE 😊


In [78]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
)

In [79]:
import json

# Check if a file is already processed
def is_file_processed_opensmile(filename, connection):
    query = "SELECT COUNT(*) FROM new_audio_features WHERE filename = %s"
    cursor = connection.cursor()
    cursor.execute(query, (filename,))
    result = cursor.fetchone()
    cursor.close()
    return result[0] > 0

# Insert feature data into MySQL with features stored in JSON
def insert_features_into_db_opensmile(filename, cel_total, features, connection):
    # Convert features dictionary to JSON format
    features_json = json.dumps(features)
    
    # Build the SQL INSERT statement
    query = """
        INSERT INTO  new_audio_features (filename, cel_total, feature_data)
        VALUES (%s, %s, %s)
    """
    
    # Execute the INSERT statement
    cursor = connection.cursor()
    cursor.execute(query, (filename, cel_total, features_json))
    connection.commit()
    cursor.close()

# Process each audio file in the merged DataFrame
connection = connect_to_db()
for idx, row in merged_df.iterrows():
    filename = row['Filename']
    
    # Check if this file has already been processed
    if is_file_processed_opensmile(filename, connection):
        print(f"Skipping {filename}, already processed.")
        continue
    
    # Get file path and preprocess the audio file (convert to WAV and resample)
    audio_path = row['Filepath']
    audio_path = convert_to_wav(audio_path)
    waveform, sample_rate = preprocess_audio(audio_path)
    
    # Process the preprocessed audio file with opensmile
    features_df = smile.process_file(audio_path)
    
    # Extract features as a dictionary
    features = features_df.iloc[0].to_dict()  # Take the first row if `process_file` returns a DataFrame
    
    # Insert the features into the database as JSON
    insert_features_into_db_opensmile(filename, row['CEL Total'], features, connection)
    print(f"Processed and stored features for {filename}")

# Close the database connection
connection.close()

Skipping 559_2021-01-22, already processed.
Skipping 66_2021-01-26, already processed.
Skipping 340_2021-01-26, already processed.
Skipping 343_2021-01-26, already processed.
Skipping 383_2021-01-27, already processed.
Skipping 606_2021-01-28, already processed.
Skipping 50_2021-01-29, already processed.
Skipping 223_2021-01-29, already processed.
Skipping 250_2021-02-01, already processed.
Skipping 628_2021-02-01, already processed.
Skipping 501_2021-02-02, already processed.
Skipping 111_2021-02-03, already processed.
Skipping 449_2021-02-03, already processed.
Skipping 512_2021-02-03, already processed.
Skipping 646_2021-02-03, already processed.
Skipping 25_2021-02-05, already processed.
Skipping 45_2021-02-05, already processed.
Skipping 527_2021-02-05, already processed.
Skipping 617_2021-02-05, already processed.
Skipping 629_2021-02-05, already processed.
Skipping 635_2021-02-05, already processed.
Skipping 636_2021-02-05, already processed.
Skipping 117_2021-02-08, already pro

In [80]:
# Retrieve data from MySQL and convert to DataFrame
def retrieve_features_from_opensmile_db():
    # Connect to the database
    connection = connect_to_db()
    
    # Query to retrieve data
    query = "SELECT filename, cel_total, feature_data FROM new_audio_features"
    
    # Execute the query and load data into a DataFrame
    df = pd.read_sql(query, connection)
    
    # Close the connection
    connection.close()
    
    # Convert the JSON feature_data column to individual columns
    # Using `pd.json_normalize()` to expand the JSON data in `feature_data`
    features_df = pd.json_normalize(df['feature_data'].apply(json.loads))
    
    # Combine the expanded features with the original columns (filename and cel_total)
    df = df.drop(columns=['feature_data']).join(features_df)
    
    return df

# Retrieve the features and print the DataFrame
opensmile_df = retrieve_features_from_opensmile_db()
opensmile_df.head

<bound method NDFrame.head of             filename  cel_total  F0semitoneFrom27.5Hz_sma3nz_amean  \
0      66_2021-01-26          3                          36.967087   
1     340_2021-01-26          2                          29.754555   
2     343_2021-01-26          1                          33.506824   
3     383_2021-01-27          5                          34.421017   
4     606_2021-01-28          4                          30.707926   
...              ...        ...                                ...   
1468  720_2023-08-29          6                          30.899710   
1469  818_2023-08-29          5                          31.289822   
1470  816_2023-08-30          2                          32.331303   
1471  526_2024-08-08          4                          30.980978   
1472  844_2024-08-30          3                          34.043789   

      F0semitoneFrom27.5Hz_sma3nz_stddevNorm  \
0                                   0.155825   
1                                

In [81]:
opensmile_result = pd.DataFrame()
opensmile_result = pd.concat([opensmile_df])
opensmile_result

,filename,cel_total,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,66_2021-01-26,3,36.967087,0.155825,33.618988,36.135857,40.558014,6.939026,285.630859,462.472565,...,0.021962,0.000588,0.007033,1.138801,1.006154,0.289442,0.275105,0.715878,1.807330,-38.858128
1,340_2021-01-26,2,29.754555,0.188660,27.937983,29.803995,32.203140,4.265158,278.571777,408.908203,...,0.021571,-0.002112,0.033466,1.855150,1.685860,0.261949,0.235339,0.337595,1.691076,-31.032339
2,343_2021-01-26,1,33.506824,0.269556,28.002808,30.514271,36.435188,8.432381,465.076996,756.858643,...,0.059314,0.001668,0.024391,0.946706,1.283908,0.184165,0.193169,0.609423,1.342605,-26.758471
3,383_2021-01-27,5,34.421017,0.146730,31.728123,33.273487,35.798702,4.070580,283.022003,465.524139,...,0.013972,0.003007,0.009543,0.784771,0.720525,0.298883,0.299312,1.130237,2.776889,-25.620874
4,606_2021-01-28,4,30.707926,0.151218,27.958063,29.963886,32.724167,4.766104,199.833069,342.037842,...,0.050015,0.001187,0.016080,1.210126,0.951087,0.292987,0.257729,0.805972,2.044664,-29.157175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,720_2023-08-29,6,30.899710,0.174081,27.429354,30.841869,34.072849,6.643496,273.285248,375.452301,...,-0.016973,0.008529,0.007863,0.939549,0.873891,0.268085,0.226813,0.944442,2.329294,-33.341125
1469,818_2023-08-29,5,31.289822,0.193646,27.932232,30.283236,34.473267,6.541035,273.565002,489.052521,...,0.008443,-0.000008,0.012496,1.335589,1.121995,0.279603,0.261781,0.630526,2.388899,-29.294714
1470,816_2023-08-30,2,32.331303,0.247999,26.689083,29.990459,36.705391,10.016308,427.653992,622.449341,...,0.010418,-0.000344,0.033759,1.460964,0.792324,0.308380,0.334101,0.362860,1.226897,-20.790131
1471,526_2024-08-08,4,30.980978,0.276731,26.091820,28.769602,32.501221,6.409401,428.575775,671.071106,...,0.021222,0.003551,0.008634,0.835132,0.959320,0.238523,0.221038,0.897157,2.399742,-31.369741


In [82]:
opensmile_result = opensmile_result.rename(columns={'filename': 'group_id'})
opensmile_result = opensmile_result.drop(columns=['cel_total'])
opensmile_result

,group_id,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,...,slopeUV0-500_sma3nz_amean,slopeUV500-1500_sma3nz_amean,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp
0,66_2021-01-26,36.967087,0.155825,33.618988,36.135857,40.558014,6.939026,285.630859,462.472565,122.483543,...,0.021962,0.000588,0.007033,1.138801,1.006154,0.289442,0.275105,0.715878,1.807330,-38.858128
1,340_2021-01-26,29.754555,0.188660,27.937983,29.803995,32.203140,4.265158,278.571777,408.908203,143.051727,...,0.021571,-0.002112,0.033466,1.855150,1.685860,0.261949,0.235339,0.337595,1.691076,-31.032339
2,343_2021-01-26,33.506824,0.269556,28.002808,30.514271,36.435188,8.432381,465.076996,756.858643,178.436264,...,0.059314,0.001668,0.024391,0.946706,1.283908,0.184165,0.193169,0.609423,1.342605,-26.758471
3,383_2021-01-27,34.421017,0.146730,31.728123,33.273487,35.798702,4.070580,283.022003,465.524139,110.145844,...,0.013972,0.003007,0.009543,0.784771,0.720525,0.298883,0.299312,1.130237,2.776889,-25.620874
4,606_2021-01-28,30.707926,0.151218,27.958063,29.963886,32.724167,4.766104,199.833069,342.037842,178.643768,...,0.050015,0.001187,0.016080,1.210126,0.951087,0.292987,0.257729,0.805972,2.044664,-29.157175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,720_2023-08-29,30.899710,0.174081,27.429354,30.841869,34.072849,6.643496,273.285248,375.452301,131.934601,...,-0.016973,0.008529,0.007863,0.939549,0.873891,0.268085,0.226813,0.944442,2.329294,-33.341125
1469,818_2023-08-29,31.289822,0.193646,27.932232,30.283236,34.473267,6.541035,273.565002,489.052521,144.182220,...,0.008443,-0.000008,0.012496,1.335589,1.121995,0.279603,0.261781,0.630526,2.388899,-29.294714
1470,816_2023-08-30,32.331303,0.247999,26.689083,29.990459,36.705391,10.016308,427.653992,622.449341,133.242935,...,0.010418,-0.000344,0.033759,1.460964,0.792324,0.308380,0.334101,0.362860,1.226897,-20.790131
1471,526_2024-08-08,30.980978,0.276731,26.091820,28.769602,32.501221,6.409401,428.575775,671.071106,201.574356,...,0.021222,0.003551,0.008634,0.835132,0.959320,0.238523,0.221038,0.897157,2.399742,-31.369741


In [83]:
melted_df = opensmile_result.melt(
    id_vars=["group_id"],  # Columns to keep as is
    var_name="feat",            # Name for the feature column
    value_name="value"          # Name for the value column
)

# Optionally, calculate `group_norm` (if needed)
melted_df["group_norm"] = melted_df["value"]

# Ensure the `id` column is auto-incremented by dropping it
#melted_df = melted_df.drop(columns=["id"])
melted_df

,group_id,feat,value,group_norm
0,66_2021-01-26,F0semitoneFrom27.5Hz_sma3nz_amean,36.967087,36.967087
1,340_2021-01-26,F0semitoneFrom27.5Hz_sma3nz_amean,29.754555,29.754555
2,343_2021-01-26,F0semitoneFrom27.5Hz_sma3nz_amean,33.506824,33.506824
3,383_2021-01-27,F0semitoneFrom27.5Hz_sma3nz_amean,34.421017,34.421017
4,606_2021-01-28,F0semitoneFrom27.5Hz_sma3nz_amean,30.707926,30.707926
...,...,...,...,...
129619,720_2023-08-29,equivalentSoundLevel_dBp,-33.341125,-33.341125
129620,818_2023-08-29,equivalentSoundLevel_dBp,-29.294714,-29.294714
129621,816_2023-08-30,equivalentSoundLevel_dBp,-20.790131,-20.790131
129622,526_2024-08-08,equivalentSoundLevel_dBp,-31.369741,-31.369741


In [84]:
connection = connect_to_db()

# Insert the DataFrame into the database
insert_feature_table_into_db(connection, melted_df, 'feat$opensmile_n$merged_data$message_id')

# Close the connection
connection.close()

129624 rows inserted into feat$opensmile_n$merged_data$message_id.


In [41]:
import mysql.connector
from mysql.connector import IntegrityError
import librosa

# Check if a file is already processed for Librosa
def is_file_processed_librosa(filename, connection):
    query = "SELECT COUNT(*) FROM new_librosa_features WHERE filename = %s"
    cursor = connection.cursor()
    cursor.execute(query, (filename,))
    result = cursor.fetchone()
    cursor.close()
    return result[0] > 0

# Helper function to convert all feature values to Python float
def convert_features_to_float(features):
    return {key: float(value) for key, value in features.items()}

# Insert Librosa feature data into MySQL with features stored in individual columns
def insert_features_into_db_librosa(filename, cel_total, features, connection):
    """
    Inserts Librosa features into the librosa_features table with separate columns.

    Parameters:
        filename (str): Name of the audio file.
        cel_total (int): CEL Total value.
        features (dict): Dictionary containing individual feature values.
        connection (mysql.connector.connection_cext.CMySQLConnection): Database connection.
    """
    # Build the SQL INSERT statement with all feature columns
    query = """
        INSERT INTO new_librosa_features (
            filename, cel_total,
            mfcc_1, mfcc_2, mfcc_3, mfcc_4, mfcc_5, mfcc_6, mfcc_7, mfcc_8, mfcc_9, mfcc_10, mfcc_11, mfcc_12, mfcc_13,
            chroma_1, chroma_2, chroma_3, chroma_4, chroma_5, chroma_6, chroma_7, chroma_8, chroma_9, chroma_10, chroma_11, chroma_12,
            spectral_contrast_1, spectral_contrast_2, spectral_contrast_3, spectral_contrast_4, spectral_contrast_5, spectral_contrast_6, spectral_contrast_7,
            tonnetz_1, tonnetz_2, tonnetz_3, tonnetz_4, tonnetz_5, tonnetz_6
        )
        VALUES (
            %s, %s,
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s
        )
    """
    
    # Prepare the parameters in the exact order of the SQL statement, converting to float
    params = (
        filename,
        cel_total,
        features.get('mfcc_1'),
        features.get('mfcc_2'),
        features.get('mfcc_3'),
        features.get('mfcc_4'),
        features.get('mfcc_5'),
        features.get('mfcc_6'),
        features.get('mfcc_7'),
        features.get('mfcc_8'),
        features.get('mfcc_9'),
        features.get('mfcc_10'),
        features.get('mfcc_11'),
        features.get('mfcc_12'),
        features.get('mfcc_13'),
        features.get('chroma_1'),
        features.get('chroma_2'),
        features.get('chroma_3'),
        features.get('chroma_4'),
        features.get('chroma_5'),
        features.get('chroma_6'),
        features.get('chroma_7'),
        features.get('chroma_8'),
        features.get('chroma_9'),
        features.get('chroma_10'),
        features.get('chroma_11'),
        features.get('chroma_12'),
        features.get('spectral_contrast_1'),
        features.get('spectral_contrast_2'),
        features.get('spectral_contrast_3'),
        features.get('spectral_contrast_4'),
        features.get('spectral_contrast_5'),
        features.get('spectral_contrast_6'),
        features.get('spectral_contrast_7'),
        features.get('tonnetz_1'),
        features.get('tonnetz_2'),
        features.get('tonnetz_3'),
        features.get('tonnetz_4'),
        features.get('tonnetz_5'),
        features.get('tonnetz_6')
    )
    
    try:
        cursor = connection.cursor()
        cursor.execute(query, params)
        connection.commit()
        cursor.close()
    except IntegrityError as e:
        print(f"IntegrityError while inserting {filename}: {e}")
        connection.rollback()
    except Exception as e:
        print(f"Unexpected error while inserting {filename}: {e}")
        connection.rollback()

# Process each audio file in the merged DataFrame using Librosa
def process_audio_files(merged_df, connection):
    for idx, row in merged_df.iterrows():
        filename = row['Filename']
        
        # Check if this file has already been processed
        if is_file_processed_librosa(filename, connection):
            print(f"Skipping {filename}, already processed.")
            continue
        
        # Get file path and load the audio file
        audio_path = row['Filepath']
        try:
            y, sr = librosa.load(audio_path, sr=None)
        except Exception as e:
            print(f"Error loading {audio_path}: {e}")
            continue
    
        # Extract features using Librosa
        try:
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            chroma = librosa.feature.chroma_stft(y=y, sr=sr)
            spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
            tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
        except Exception as e:
            print(f"Error extracting features for {filename}: {e}")
            continue
    
        # Aggregate features by taking the mean across time frames
        mfccs_mean = mfccs.mean(axis=1)
        chroma_mean = chroma.mean(axis=1)
        spectral_contrast_mean = spectral_contrast.mean(axis=1)
        tonnetz_mean = tonnetz.mean(axis=1)
    
        # Create a features dictionary with individual feature values
        features = {
            "mfcc_1": mfccs_mean[0],
            "mfcc_2": mfccs_mean[1],
            "mfcc_3": mfccs_mean[2],
            "mfcc_4": mfccs_mean[3],
            "mfcc_5": mfccs_mean[4],
            "mfcc_6": mfccs_mean[5],
            "mfcc_7": mfccs_mean[6],
            "mfcc_8": mfccs_mean[7],
            "mfcc_9": mfccs_mean[8],
            "mfcc_10": mfccs_mean[9],
            "mfcc_11": mfccs_mean[10],
            "mfcc_12": mfccs_mean[11],
            "mfcc_13": mfccs_mean[12],
            "chroma_1": chroma_mean[0],
            "chroma_2": chroma_mean[1],
            "chroma_3": chroma_mean[2],
            "chroma_4": chroma_mean[3],
            "chroma_5": chroma_mean[4],
            "chroma_6": chroma_mean[5],
            "chroma_7": chroma_mean[6],
            "chroma_8": chroma_mean[7],
            "chroma_9": chroma_mean[8],
            "chroma_10": chroma_mean[9],
            "chroma_11": chroma_mean[10],
            "chroma_12": chroma_mean[11],
            "spectral_contrast_1": spectral_contrast_mean[0],
            "spectral_contrast_2": spectral_contrast_mean[1],
            "spectral_contrast_3": spectral_contrast_mean[2],
            "spectral_contrast_4": spectral_contrast_mean[3],
            "spectral_contrast_5": spectral_contrast_mean[4],
            "spectral_contrast_6": spectral_contrast_mean[5],
            "spectral_contrast_7": spectral_contrast_mean[6],
            "tonnetz_1": tonnetz_mean[0],
            "tonnetz_2": tonnetz_mean[1],
            "tonnetz_3": tonnetz_mean[2],
            "tonnetz_4": tonnetz_mean[3],
            "tonnetz_5": tonnetz_mean[4],
            "tonnetz_6": tonnetz_mean[5]
        }
        
        # Convert feature values to Python floats
        features = convert_features_to_float(features)
    
        # Insert the features into the database as individual columns
        insert_features_into_db_librosa(filename, row['CEL Total'], features, connection)
        print(f"Processed and stored features for {filename}")

# Example usage
if __name__ == "__main__":
    try:
        connection = connect_to_db()  # Ensure this function is correctly defined
        process_audio_files(merged_df, connection)
    finally:
        connection.close()


Skipping 559_2021-01-22, already processed.
Skipping 66_2021-01-26, already processed.
Skipping 340_2021-01-26, already processed.
Skipping 343_2021-01-26, already processed.
Skipping 383_2021-01-27, already processed.
Skipping 606_2021-01-28, already processed.
Skipping 50_2021-01-29, already processed.
Skipping 223_2021-01-29, already processed.
Skipping 250_2021-02-01, already processed.
Skipping 628_2021-02-01, already processed.
Skipping 501_2021-02-02, already processed.
Skipping 111_2021-02-03, already processed.
Skipping 449_2021-02-03, already processed.
Skipping 512_2021-02-03, already processed.
Skipping 646_2021-02-03, already processed.
Skipping 25_2021-02-05, already processed.
Skipping 45_2021-02-05, already processed.
Skipping 527_2021-02-05, already processed.
Skipping 617_2021-02-05, already processed.
Skipping 629_2021-02-05, already processed.
Skipping 635_2021-02-05, already processed.
Skipping 636_2021-02-05, already processed.
Skipping 117_2021-02-08, already pro

### Librosa


In [42]:

# Retrieve data from MySQL and convert to DataFrame
def retrieve_features_from_librosa_db():
    # Connect to the database
    connection = connect_to_db()
    
    # Query to retrieve data
    query = "SELECT * FROM new_librosa_features"
    
    # Execute the query and load data into a DataFrame
    df = pd.read_sql(query, connection)
    print(df.shape)
    
    # Close the connection
    connection.close()
        
    return df

# Retrieve the features and print the DataFrame
librosa_df = retrieve_features_from_librosa_db()
print(librosa_df.head())


(1473, 42)
   id        filename  cel_total   mfcc_1   mfcc_2    mfcc_3    mfcc_4  \
0   1   66_2021-01-26          3 -547.983  50.4426 -14.86450  -2.39750   
1   2  340_2021-01-26          2 -453.576  98.7346 -31.00630  -5.23638   
2   3  343_2021-01-26          1 -453.458  61.0583 -33.28480   5.95882   
3   4  383_2021-01-27          5 -465.388  39.8266  -4.30423   3.84766   
4   5  606_2021-01-28          4 -458.748  83.0511 -23.53380 -13.53490   

     mfcc_5    mfcc_6     mfcc_7  ...  spectral_contrast_5  \
0   2.34822 -15.61700  -9.959200  ...              16.2824   
1   7.75963 -26.83480 -14.183700  ...              17.7849   
2  12.93660 -21.27960  -5.896230  ...              16.5694   
3   6.37563  -4.97029  -0.780105  ...              15.6146   
4  -1.47376 -20.66810 -10.793700  ...              15.4339   

   spectral_contrast_6  spectral_contrast_7  tonnetz_1  tonnetz_2  tonnetz_3  \
0              47.2366              31.9090  -0.005635   0.005738   0.001151   
1          

In [43]:
librosa_result = pd.DataFrame()
librosa_result = pd.concat([librosa_df])
librosa_result

,id,filename,cel_total,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,...,spectral_contrast_5,spectral_contrast_6,spectral_contrast_7,tonnetz_1,tonnetz_2,tonnetz_3,tonnetz_4,tonnetz_5,tonnetz_6,created_at
0,1,66_2021-01-26,3,-547.983,50.4426,-14.86450,-2.397500,2.348220,-15.61700,-9.959200,...,16.2824,47.2366,31.9090,-0.005635,0.005738,0.001151,-0.000925,0.003331,-0.004590,2025-01-03 01:27:01
1,2,340_2021-01-26,2,-453.576,98.7346,-31.00630,-5.236380,7.759630,-26.83480,-14.183700,...,17.7849,50.0224,25.6802,-0.003651,0.002260,-0.008349,-0.028935,-0.002668,-0.000746,2025-01-03 01:30:58
2,3,343_2021-01-26,1,-453.458,61.0583,-33.28480,5.958820,12.936600,-21.27960,-5.896230,...,16.5694,57.9770,19.3772,-0.002980,0.006770,-0.001244,-0.015849,-0.005023,-0.000099,2025-01-03 01:31:44
3,4,383_2021-01-27,5,-465.388,39.8266,-4.30423,3.847660,6.375630,-4.97029,-0.780105,...,15.6146,49.2063,32.3530,0.002044,0.007342,-0.012093,-0.002854,0.004781,-0.001256,2025-01-03 01:32:14
4,5,606_2021-01-28,4,-458.748,83.0511,-23.53380,-13.534900,-1.473760,-20.66810,-10.793700,...,15.4339,55.2542,19.7222,-0.002247,-0.000322,-0.014318,-0.009615,0.000540,-0.003135,2025-01-03 01:32:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468,1469,941934_2024-11-07,1,-523.056,64.5252,4.41419,-2.658930,0.741109,-6.20757,-6.083630,...,16.3449,29.9292,36.8315,-0.008267,0.006402,-0.023331,0.003684,-0.001424,-0.000680,2025-03-07 14:20:35
1469,1470,952421_2024-11-19,5,-466.212,79.3642,-11.80360,12.666300,10.887700,-21.75930,-6.114550,...,16.5389,39.7339,31.9168,-0.004559,0.007508,-0.010312,0.004298,0.003147,0.002484,2025-03-07 14:21:43
1470,1471,981476_2024-11-01,2,-484.339,66.3646,-2.71387,6.896240,10.244300,-6.55915,-0.109896,...,15.5608,35.8146,34.5245,-0.005835,-0.001265,-0.018404,0.003864,0.005158,0.000599,2025-03-07 14:22:44
1471,1472,983777_2024-11-22,2,-465.480,75.7691,-9.81335,0.418301,11.481600,-10.59200,-15.171200,...,20.0144,39.2793,33.7522,-0.005838,0.008339,-0.005554,0.002615,-0.003948,-0.000014,2025-03-07 14:23:56


In [44]:
librosa_result = librosa_result.rename(columns={'filename': 'group_id'})
librosa_result = librosa_result.drop(columns=['cel_total','created_at'])
print(librosa_result)

        id           group_id   mfcc_1   mfcc_2    mfcc_3     mfcc_4  \
0        1      66_2021-01-26 -547.983  50.4426 -14.86450  -2.397500   
1        2     340_2021-01-26 -453.576  98.7346 -31.00630  -5.236380   
2        3     343_2021-01-26 -453.458  61.0583 -33.28480   5.958820   
3        4     383_2021-01-27 -465.388  39.8266  -4.30423   3.847660   
4        5     606_2021-01-28 -458.748  83.0511 -23.53380 -13.534900   
...    ...                ...      ...      ...       ...        ...   
1468  1469  941934_2024-11-07 -523.056  64.5252   4.41419  -2.658930   
1469  1470  952421_2024-11-19 -466.212  79.3642 -11.80360  12.666300   
1470  1471  981476_2024-11-01 -484.339  66.3646  -2.71387   6.896240   
1471  1472  983777_2024-11-22 -465.480  75.7691  -9.81335   0.418301   
1472  1473  997606_2024-10-25 -578.940  48.5309   3.36018   7.491950   

         mfcc_5    mfcc_6     mfcc_7     mfcc_8  ...  spectral_contrast_4  \
0      2.348220 -15.61700  -9.959200  -5.127280  ...      

In [45]:
melted_df = librosa_result.melt(
    id_vars=["id", "group_id"],  # Columns to keep as is
    var_name="feat",            # Name for the feature column
    value_name="value"          # Name for the value column
)

# Optionally, calculate `group_norm` (if needed)
melted_df["group_norm"] = melted_df["value"]

# Ensure the `id` column is auto-incremented by dropping it
melted_df = melted_df.drop(columns=["id"])

print(melted_df.head(100))

          group_id    feat    value  group_norm
0    66_2021-01-26  mfcc_1 -547.983    -547.983
1   340_2021-01-26  mfcc_1 -453.576    -453.576
2   343_2021-01-26  mfcc_1 -453.458    -453.458
3   383_2021-01-27  mfcc_1 -465.388    -465.388
4   606_2021-01-28  mfcc_1 -458.748    -458.748
..             ...     ...      ...         ...
95  523_2021-05-10  mfcc_1 -436.193    -436.193
96  644_2021-05-10  mfcc_1 -298.543    -298.543
97  645_2021-05-10  mfcc_1 -365.570    -365.570
98   61_2021-05-11  mfcc_1 -504.679    -504.679
99  387_2021-05-11  mfcc_1 -452.279    -452.279

[100 rows x 4 columns]


In [48]:
connection = connect_to_db()

# Insert the DataFrame into the database
insert_feature_table_into_db(connection, melted_df, 'feat$librosa_n$merged_data$message_id')

# Close the connection
connection.close()

55974 rows inserted into feat$librosa_n$merged_data$message_id.


In [ ]:
# # Filter records for a specific group_id
# specific_group_id = "66_2021-01-26"
# filtered_df = melted_df[melted_df["group_id"] == specific_group_id]

# # Display all records for the specific group_id
# filtered_df

# CREATE TABLE feat$whisper_mean$merged_data$message_id (
#     id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT,
#     group_id VARCHAR(100) NULL,
#     feat VARCHAR(36) NULL,
#     value INT(11) NULL,
#     group_norm DOUBLE NULL,
#     PRIMARY KEY (id),
#     KEY group_id (group_id),
#     KEY feat (feat)
# );

# CREATE TABLE feat$whisper_median$merged_data$message_id (
#     id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT,
#     group_id VARCHAR(100) NULL,
#     feat VARCHAR(36) NULL,
#     value INT(11) NULL,
#     group_norm DOUBLE NULL,
#     PRIMARY KEY (id),
#     KEY group_id (group_id),
#     KEY feat (feat)
# );


# CREATE TABLE feat$trill_mstd$merged_data$message_id (
#     id BIGINT(16) UNSIGNED NOT NULL AUTO_INCREMENT,
#     group_id VARCHAR(100) NULL,
#     feat VARCHAR(36) NULL,
#     value INT(11) NULL,
#     group_norm DOUBLE NULL,
#     PRIMARY KEY (id),
#     KEY group_id (group_id),
#     KEY feat (feat)
# );


### Trill path

In [115]:
directory_path = '/sandata/karthik9/mstd_embeddings/'

# Use glob to get all files ending with "__wb_mean_trill_embedding.npy"
pattern = os.path.join(directory_path, "*_std_trill_embedding.npy")
file_list = glob.glob(pattern)


# Create a dictionary mapping from filename (without suffix) to its full path
filename_to_path = {}
for file_path in file_list:
    base_name = os.path.basename(file_path)
    # Extract the part before "__wb_mean_trill_embedding.npy"
    key = base_name.split('_std_trill_embedding.npy')[0]
    filename_to_path[key] = file_path



print(filename_to_path)

# Add a new column to your DataFrame by mapping the "filename" column
# to the corresponding file path from our dictionary
stopwords_merged_df['Original_Filename'] = stopwords_merged_df['Original_Filename'].str.replace(r'\.mp3$', '', regex=True)

stopwords_merged_df['Trill path'] = stopwords_merged_df['Original_Filename'].map(filename_to_path)

# Optionally, check which rows didn't find a match
missing = stopwords_merged_df[stopwords_merged_df['Trill path'].isna()]
if not missing.empty:
    print("No matching file found for these filenames:")
    print(missing['Original_Filename'])
else:
    print("All filenames matched a file path.")

{'1007_03-10-2021_12-50pm_wb': '/sandata/karthik9/mstd_embeddings/1007_03-10-2021_12-50pm_wb_std_trill_embedding.npy', '107_04-20-2021_14-01pm_wb': '/sandata/karthik9/mstd_embeddings/107_04-20-2021_14-01pm_wb_std_trill_embedding.npy', '111_04-28-2021_16-34pm_wb': '/sandata/karthik9/mstd_embeddings/111_04-28-2021_16-34pm_wb_std_trill_embedding.npy', '108_04-30-2021_18-29pm_wb': '/sandata/karthik9/mstd_embeddings/108_04-30-2021_18-29pm_wb_std_trill_embedding.npy', '105_04-29-2021_20-18pm_wb': '/sandata/karthik9/mstd_embeddings/105_04-29-2021_20-18pm_wb_std_trill_embedding.npy', '109_02-23-2021_15-00pm_wb': '/sandata/karthik9/mstd_embeddings/109_02-23-2021_15-00pm_wb_std_trill_embedding.npy', '117_06-01-2021_12-16pm_wb': '/sandata/karthik9/mstd_embeddings/117_06-01-2021_12-16pm_wb_std_trill_embedding.npy', '114_06-07-2021_17-21pm_wb': '/sandata/karthik9/mstd_embeddings/114_06-07-2021_17-21pm_wb_std_trill_embedding.npy', '107_06-01-2021_14-03pm_wb': '/sandata/karthik9/mstd_embeddings/107_0

In [116]:
stopwords_merged_df

,New ID,Filename,Text,CEL Total,CELVAL1,CELVAL2,CELVAL3,CEL1,CEL2,CEL3,...,Monthly Income,Cognitive Impaired,Dementia,KlaatchID,Date,Filepath,Cleaned Name,Original_Filename,WordCount,Trill path
0,938730,66_2021-01-26,Hello. Please leave a message after the tone. ...,3,1,1,1,Agree,Agree,Agree,...,1841.96,NaN,YES,66,2021-01-26,/sandata/karthik9/nwa/New_audio/66_01-26-2021_...,66_01-26-2021_15-20min,66_01-26-2021_15-20min_wb,353,/sandata/karthik9/mstd_embeddings/66_01-26-202...
1,938936,340_2021-01-26,"Yes. I'm fine, how are you? Ok. Well, what I u...",2,1,0,1,Agree,Strongly Agree,Agree,...,1630.00,NaN,NaN,340,2021-01-26,/sandata/karthik9/nwa/New_audio/340_01-26-2021...,340_01-26-2021_15-05min,340_01-26-2021_15-05min_wb,916,/sandata/karthik9/mstd_embeddings/340_01-26-20...
2,NaN,343_2021-01-26,"Hello. All right. I'll do it now. Sure. No, no...",1,1,0,0,Agree,Strongly Agree,Strongly Agree,...,NaN,NaN,NaN,343,2021-01-26,/sandata/karthik9/nwa/New_audio/343_01-26-2021...,343_01-26-2021_08-43min,343_01-26-2021_08-43min_wb,358,/sandata/karthik9/mstd_embeddings/343_01-26-20...
3,938879,383_2021-01-27,"Hello. Hi, I'm. Ok. OK. Yes, okay. I wake up, ...",5,1,3,1,Agree,Disagree,Agree,...,885.00,NaN,NaN,383,2021-01-27,/sandata/karthik9/nwa/New_audio/383_01-27-2021...,383_01-27-2021_15-35min,383_01-27-2021_15-35min_wb,132,/sandata/karthik9/mstd_embeddings/383_01-27-20...
4,938733,606_2021-01-28,"Hello. Uh huh. Yeah. Ok. OK. Well, yesterday, ...",4,1,0,3,Agree,Strongly Agree,Disagree,...,1863.25,NaN,NaN,606,2021-01-28,/sandata/karthik9/nwa/New_audio/606_01-28-2021...,606_01-28-2021_12-04min,606_01-28-2021_12-04min_wb,242,/sandata/karthik9/mstd_embeddings/606_01-28-20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,938947,598_2024-09-27,ello. Hello. Hi. Good morning. Good morning. :...,3,1,1,1,Agree,Agree,Agree,...,1497.00,NaN,NaN,598,2024-09-27,/sandata/karthik9/nwa/New_audio/R_The_towers_5...,598_9-27-2024_9-41am,R_The_towers_598_9-27-2024_9-41am_wb,1272,/sandata/karthik9/mstd_embeddings/R_The_towers...
1350,938945,45_2024-09-27,"ello. I know, yeah. Hello? Hey. I'm hanging in...",5,1,2,2,Agree,Neither Agree or Disagree,Neither Agree or Disagree,...,1979.10,NaN,NaN,45,2024-09-27,/sandata/karthik9/nwa/New_audio/R_The_towers_4...,45_9-27-2024_10-01am,R_The_towers_45_9-27-2024_10-01am_wb,1208,/sandata/karthik9/mstd_embeddings/R_The_towers...
1351,938730,66_2024-09-27,ello. Please leave a message after the tone. :...,3,1,1,1,Agree,Agree,Agree,...,1841.96,NaN,YES,66,2024-09-27,/sandata/karthik9/nwa/New_audio/R_The_towers_6...,66_9-27-2024_10-04am,R_The_towers_66_9-27-2024_10-04am_wb,780,/sandata/karthik9/mstd_embeddings/R_The_towers...
1352,938764,73_2024-09-30,ello. : Hi. : That's okay. Because everyone I'...,1,1,0,0,Agree,Strongly Agree,Strongly Agree,...,1777.00,NaN,NaN,73,2024-09-30,/sandata/karthik9/nwa/New_audio/R_The_towers_7...,73_9-30-2024_10-31am,R_The_towers_73_9-30-2024_10-31am_wb,1961,/sandata/karthik9/mstd_embeddings/R_The_towers...


In [117]:
trill_mean = stopwords_merged_df[['Filename','Trill path','KlaatchID']]
trill_mean

,Filename,Trill path,KlaatchID
0,66_2021-01-26,/sandata/karthik9/mstd_embeddings/66_01-26-202...,66
1,340_2021-01-26,/sandata/karthik9/mstd_embeddings/340_01-26-20...,340
2,343_2021-01-26,/sandata/karthik9/mstd_embeddings/343_01-26-20...,343
3,383_2021-01-27,/sandata/karthik9/mstd_embeddings/383_01-27-20...,383
4,606_2021-01-28,/sandata/karthik9/mstd_embeddings/606_01-28-20...,606
...,...,...,...
1349,598_2024-09-27,/sandata/karthik9/mstd_embeddings/R_The_towers...,598
1350,45_2024-09-27,/sandata/karthik9/mstd_embeddings/R_The_towers...,45
1351,66_2024-09-27,/sandata/karthik9/mstd_embeddings/R_The_towers...,66
1352,73_2024-09-30,/sandata/karthik9/mstd_embeddings/R_The_towers...,73


In [120]:
# Assuming stopwords_merged_df is already loaded
trill_mean = stopwords_merged_df[['Filename', 'Trill path', 'KlaatchID']].copy()

# Function to load numpy array and expand it into a dictionary of columns
def load_numpy_to_columns(npy_path):
    array = np.load(npy_path)  # Load the .npy file
    return {f"dim{i}": array[i] for i in range(1024)}  # Expand into columns

# Apply the function to each row and expand columns
expanded_columns = trill_mean['Trill path'].apply(load_numpy_to_columns).apply(pd.Series)

# Concatenate original DataFrame with expanded columns
trill_mean = pd.concat([trill_mean, expanded_columns], axis=1)

# Drop the 'Trill path' column if no longer needed
trill_mean.drop(columns=['Trill path','KlaatchID'], inplace=True)
trill_mean = trill_mean.rename(columns={'Filename': 'group_id'})

In [121]:
trill_mean

,group_id,dim0,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,...,dim1014,dim1015,dim1016,dim1017,dim1018,dim1019,dim1020,dim1021,dim1022,dim1023
0,66_2021-01-26,0.161751,0.342980,0.402845,0.241190,0.385969,2.301883,0.300886,0.177819,0.186381,...,0.446909,0.303238,0.291450,0.224259,0.233643,0.254942,0.198047,0.187553,0.240241,0.249406
1,340_2021-01-26,0.151842,0.244564,0.404970,0.452595,0.387624,2.829696,0.148444,0.215397,0.209205,...,0.464960,0.244634,0.467094,0.192802,0.250561,0.223547,0.186222,0.381661,0.295561,0.392814
2,343_2021-01-26,0.151773,0.205098,0.493110,0.368905,0.326852,1.544853,0.185844,0.253989,0.347040,...,0.400705,0.258418,0.133133,0.305992,0.220930,0.192693,0.165578,0.202520,0.406406,0.277780
3,383_2021-01-27,0.141098,0.259953,0.241479,0.135106,0.246173,1.117294,0.210041,0.143299,0.170108,...,0.478077,0.221422,0.342895,0.221020,0.258630,0.205665,0.140503,0.285308,0.314400,0.302498
4,606_2021-01-28,0.090885,0.195255,0.343297,0.210879,0.280706,1.508404,0.259976,0.268952,0.163434,...,0.431528,0.188734,0.218336,0.178846,0.196685,0.221915,0.201548,0.383765,0.298072,0.390492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,598_2024-09-27,0.106653,0.223451,0.258562,0.280345,0.180008,1.673212,0.185591,0.225936,0.255307,...,0.276136,0.275548,0.274376,0.244210,0.138134,0.274895,0.202188,0.226296,0.209568,0.230032
1350,45_2024-09-27,0.112979,0.280140,0.227360,0.190409,0.241684,1.497911,0.291863,0.351002,0.250249,...,0.414915,0.432555,0.256435,0.219556,0.233612,0.310884,0.278310,0.599484,0.469348,0.366337
1351,66_2024-09-27,0.149175,0.248441,0.326305,0.233778,0.324068,1.768303,0.229997,0.209705,0.216194,...,0.404439,0.202232,0.303827,0.193031,0.245565,0.273218,0.209496,0.204436,0.215210,0.212283
1352,73_2024-09-30,0.125030,0.201882,0.222720,0.226415,0.245648,1.011064,0.227631,0.191092,0.274645,...,0.465466,0.212683,0.305655,0.238751,0.165780,0.279899,0.201164,0.279268,0.262253,0.183903


In [122]:
trill_melted_df = trill_mean.melt(
    id_vars=["group_id"],  # Columns to keep as is
    var_name="feat",            # Name for the feature column
    value_name="value"          # Name for the value column
)

# Optionally, calculate `group_norm` (if needed)
trill_melted_df["group_norm"] = trill_melted_df["value"]


trill_melted_df

,group_id,feat,value,group_norm
0,66_2021-01-26,dim0,0.161751,0.161751
1,340_2021-01-26,dim0,0.151842,0.151842
2,343_2021-01-26,dim0,0.151773,0.151773
3,383_2021-01-27,dim0,0.141098,0.141098
4,606_2021-01-28,dim0,0.090885,0.090885
...,...,...,...,...
1386491,598_2024-09-27,dim1023,0.230032,0.230032
1386492,45_2024-09-27,dim1023,0.366337,0.366337
1386493,66_2024-09-27,dim1023,0.212283,0.212283
1386494,73_2024-09-30,dim1023,0.183903,0.183903


In [ ]:
connection = connect_to_db()

# Insert the DataFrame into the database
#insert_feature_table_into_db(connection, trill_melted_df, 'feat$trill_mstd$merged_data$message_id')

# Close the connection
connection.close()

1386496 rows inserted into feat$trill_mstd$merged_data$message_id.


In [51]:
import numpy as np

# Path to the NumPy file
file_path = "/sandata/karthik9/mean_embeddings/T_774_7-25-2023_3-01pm_wb_mean_trill_embedding.npy"

# Load the NumPy array
array = np.load(file_path)

# Print the shape (dimensions) of the array
print("Array Shape:", array.shape)
print(array)


Array Shape: (1024,)
[-0.681237    0.04380174  0.55346364 ...  0.13927267 -0.24449825
 -0.29902354]


In [ ]:

# def extract_wav2vec2_features(waveform, sample_rate, chunk_duration=5.0):
#     processor = Wav2Vec2Processor.from_pretrained('facebook/wav2vec2-base-960h')
#     model = Wav2Vec2Model.from_pretrained('facebook/wav2vec2-base-960h')
#     model.eval()
    
#     chunk_size = int(chunk_duration * sample_rate)
#     total_samples = waveform.shape[1]
#     all_features = []

#     for start in range(0, total_samples, chunk_size):
#         end = min(start + chunk_size, total_samples)
#         chunk_waveform = waveform[:, start:end]
#         inputs = processor(chunk_waveform.squeeze().numpy(), sampling_rate=sample_rate, return_tensors="pt", padding=True)
#         with torch.no_grad():
#             features = model(**inputs).last_hidden_state
#             all_features.append(features.squeeze(0).numpy())

#     features_np = np.concatenate(all_features, axis=0)
#     return features_np


# # Function to check if a file is already processed
# def is_file_processed_w2v(connection, filename):
#     cursor = connection.cursor()
    
#     # Query to check if the filename exists in the database
#     query = """
#     SELECT COUNT(*) FROM vector_audio_features WHERE filename = %s
#     """
#     cursor.execute(query, (filename,))
    
#     # Fetch the count of rows
#     result = cursor.fetchone()
#     return result[0] > 0  # If count > 0, the file has already been processed



# # Function to insert the file path into the database
# def insert_file_path_into_db(connection, filename, features_file_path):
#     cursor = connection.cursor()
    
#     # Insert the file path and filename into the database
#     query = """
#     INSERT INTO vector_audio_features (filename, features_file_path)
#     VALUES (%s, %s)
#     """
#     cursor.execute(query, (filename, features_file_path))
#     connection.commit()



# # Function to save extracted features to a file
# def save_features_to_file(features, filename):
#     # Ensure the directory exists
#     if not os.path.exists(FEATURES_DIR):
#         os.makedirs(FEATURES_DIR)
    
#     # Create the full file path
#     features_file_path = os.path.join(FEATURES_DIR, f"{filename}_features.npy")
    
#     # Save the features as a NumPy file
#     np.save(features_file_path, features)
    
#     return features_file_path


# # Process the dataset and insert file paths into the database
# def process_and_store_features(merged_df):
#     connection = connect_to_db()

#     for index, row in merged_df.iterrows():
#         audio_file_path = row['Filepath']
#         filename = row['Filename']
        
#         # Check if the file is already processed
#         if is_file_processed_w2v(connection, filename):
#             print(f"File {filename} has already been processed. Skipping.")
#             continue  # Skip this file if it's already processed
        
#         try:
#             # Preprocess and extract features
#             waveform, sample_rate = preprocess_audio(audio_file_path)
#             wav2vec2_features = extract_wav2vec2_features(waveform, sample_rate, chunk_duration=30)
            
#             # Print the size (shape) of the feature vectors
#             print(f"Feature vector size for {filename}: {wav2vec2_features.shape}")
            
#             # Save the extracted features to a file
#             features_file_path = save_features_to_file(wav2vec2_features, filename)
            
#             # Store the file path in the database
#             insert_file_path_into_db(connection, filename, features_file_path)
#             print(f"Features for {filename} stored successfully in {features_file_path}.")

#         except Exception as e:
#             print(f"Error processing {audio_file_path}: {e}")

#     connection.close()

# # Usage
# process_and_store_features(merged_df)